In [1]:
import src.dataloader
import src.loss
import src.transforms as t
import src.functional
from src.models.HCNet import HCNet
from src.models.RDCNet import RDCNet

import torch
import torch.nn
from torch.utils.data import DataLoader

import time
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms
from torch.utils.tensorboard import SummaryWriter
from scipy.ndimage.morphology import binary_fill_holes, binary_erosion, binary_dilation
import skimage.io as io
from importlib import reload  

In [ ]:
reload(src.dataloader)

In [10]:
model = torch.jit.script(HCNet(in_channels=1, out_channels=4, complexity=15)).cuda()
model.train()
#model.load_state_dict(torch.load('Dec_23_test.hcnet'))
#model.load_state_dict(torch.load('Dec_17_REALLY_GOOD.hcnet'))
print('')

In [11]:
print('Loading Train...')
transforms = torchvision.transforms.Compose([
    t.nul_crop(rate=1),
    t.random_crop(shape=(256, 256, 16)),
    #t.elastic_deformation(grid_shape=(3, 3, 2), scale=1.1),
    t.colormask_to_mask(),
    t.adjust_centroids(),
    t.to_cuda(),    
    #t.random_h_flip(),
    #t.random_v_flip(),
    #t.random_affine(shear=(-5, 5), scale=(0.9, 1.5)),
    #t.gaussian_blur(rate=0.5),
    #t.adjust_brightness(range_brightness = (-0.1, 0.3)),
    #t.adjust_centroids(),
])
data = src.dataloader.em_dataset('/media/DataStorage/Dropbox (Partners HealthCare)/HairCellInstance/data/mito_train/', transforms=transforms, random_ind=True)
dl = DataLoader(data, batch_size=1, shuffle=False, num_workers=0)
print('Done')



Loading Train...
Done


In [ ]:
for dd in dl:
    print(dd['masks'].shape)

val, im = torch.max(dd['masks'], axis=1)

plt.imshow(im[0,:,:,8].cpu().numpy())

In [12]:
lr = 1e-6
gamma = 1
wd = .0000001
#sigma= 0.1 
#sigma = lambda e: 0.0231 + 0.075 * 15/(15+e)
iterations=5

In [13]:
try:
    if e > 0:
        pass
except:
    epochs = 500
    e = -1

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = wd)
loss_fun = src.loss.tversky_loss()

In [15]:
optimizer.weight_decay = wd
optimizer.lr = lr
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma, last_epoch=-1, verbose=False)

In [8]:
try:
    save_file = 'Jan12_1_mito.hcnet'
    writer
    
except:
    writer = SummaryWriter()
    best_train_loss = 2
    best_val_loss = 2


In [16]:
sigma = torch.tensor([0.01]).cuda()  
while True:
    e += 1
    epoch_loss = []
    for data_dict in dl:
        image = data_dict['image']

        image = (image - 0.5) / 0.5
        mask = data_dict['masks'] > 0.5
        centroids = data_dict['centroids']
        print(centroids.max())
        
        assert not torch.any(torch.isnan(image))
        assert not torch.any(torch.isnan(mask))

        if centroids.shape[1] == 0:
            continue

        optimizer.zero_grad()

        out = model(image.cuda(),iterations)
        
        print(out.max())
        
        prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
        out = out[:, 0:3:1, ...]
        out = src.functional.vector_to_embedding(out)
        out = src.functional.embedding_to_probability(out, centroids.cuda(), sigma)
        loss = loss_fun(out, mask.cuda()) \
               + loss_fun(prob_map, mask.cuda().sum(1).unsqueeze(1))


        loss.backward()
        optimizer.step()
        

        epoch_loss.append(loss.detach().cpu().item())

    writer.add_scalar('Loss/train', torch.mean(torch.tensor(epoch_loss)).item(), e)
    writer.add_scalar('Hyperparam/lr',scheduler.get_lr()[0],e)
    writer.add_scalar('Hyperparam/weight_decay',wd,e)
    writer.add_scalar('Hyperparam/iter',iterations,e)

    scheduler.step()
    data.step()
        

    if torch.mean(torch.tensor(epoch_loss)).item() < best_train_loss:
        torch.save(model.state_dict(), 'train_' + save_file)
        best_train_loss = torch.mean(torch.tensor(epoch_loss)).item()


tensor(4.0911, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(nan, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(nan, device='cuda:0', grad_fn=<MaxBackward1>)


KeyboardInterrupt: 

In [ ]:
del  image, mask, out, sigma

In [ ]:
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size(), obj.device)
    except:
        pass

In [ ]:
centroids.numel()

In [ ]:
#####plt.imshow(out.detach().cpu().numpy()[0,[3],:,:,6].transpose((1,2,0)))
render, values = out.max(1)

plt.imshow(values[0,:,:,15].detach().cpu().numpy())
plt.show()

values.max()

In [ ]:
save_name = 'Dec_23_test.hcnet'
torch.save(model.state_dict(), save_name)

In [ ]:
model.train()
sigma = torch.tensor([0.006]).cuda()  
with torch.no_grad():
    for data_dict in dl:
        image = data_dict['image']
        image = (image - 0.5) / 0.5
        mask = data_dict['masks'] > 0.5
        
        centroids = data_dict['centroids']
        plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
        plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
        plt.show()
        
        out = model(image.cuda(),iterations)
        prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
        out = out[:, 0:3:1, ...]
        out = src.functional.vector_to_embedding(out)
        out = src.functional.embedding_to_probability(out, centroids.cuda(), sigma)
        
        plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
        plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
        plt.show()
        
        print(loss)
        break
model.train()
print(' ')
test = out.clone()

print(loss_fun(test, mask.cuda()))

value, ind = out.max(1)
ind[prob_map[:,0,...] < 0.5]=0
#for i in range(render.shape[0]):
#    render[i,:,:,:] = binary_dilation(binary_fill_holes(render[i,:,:,:]))
ind = ind.cpu().detach().squeeze(0).float().numpy().transpose((2,1,0))
io.imsave('bigtest.tif', ind)
ind.min()


for data_dict in dl:
    plt.imshow(data_dict['masks'][0,:,:,:,:].sum(0).sum(-1).cpu())
    plt.plot(data_dict['centroids'][0,:,1].cpu(), data_dict['centroids'][0,:,0].cpu(), 'bo')
    plt.show()

Naieve Test

In [ ]:
sigma = torch.tensor([0.016]).cuda()  
with torch.no_grad():
    image = io.imread('/media/DataStorage/Dropbox (Partners HealthCare)/HairCellInstance/data/test/shit2.tif')
    image = torch.from_numpy(image[..., [0, 2, 3]] / 2 ** 16).unsqueeze(0).float()
    image = image.transpose(1, 3).transpose(0, -1).squeeze().unsqueeze(0).sub(0.5).div(0.5).cuda()


    out = model(image.cuda(), 5)
    prob_map = torch.sigmoid(out[:, -1, ...]).unsqueeze(1)
    out = out[:, 0:3:1, ...]
    embed = src.functional.vector_to_embedding(out)
    #for i in range(3):
    #    embed[:,i,...][prob_map.squeeze(0) < 0.5] = -10
    centroids = src.functional.estimate_centroids(embed, 0.004, 40)
    embed = src.functional.vector_to_embedding(out)
    out = src.functional.embedding_to_probability(embed, centroids.cuda(), sigma)

        
        
model.train()
value, ind = out.max(1)
ind[prob_map[:,0,...] < 0.75]=0

x = embed.detach().cpu().numpy()[0,0,...].flatten()
y = embed.detach().cpu().numpy()[0,1,...].flatten()
plt.figure(figsize=(10,10))
plt.hist2d(x,y,bins=512, range=((0,1), (0,1)))
plt.plot(centroids[0,:,0].detach().cpu().numpy(), centroids[0,:,1].detach().cpu().numpy(), 'ro')
# plt.plot(data_dict['centroids'][0,:,0].cpu(), data_dict['centroids'][0,:,1].cpu(), 'bo', alpha=0.5)
plt.show()

print(image.shape)
plt.imshow(image[0,:,:,:,7].mul(0.5).add(0.5).cpu().numpy().transpose((1,2,0)))
plt.show()

print(centroids.shape[1], ' cells predicted')
print(prob_map.max())

value, ind = out.max(1)
ind[value < 0.2] = 0
ind[prob_map.squeeze(1)<0.5]=0
# correction_matrix = binary_dilation(binary_erosion(ind))
# ind[np.logical_not(binary_dilation)] = 0
io.imsave('naieve_test.tif', ind.cpu().detach().squeeze(0).float().numpy().transpose((2,1,0)))
ind.min()
ind.shape



In [ ]:
prob_map = prob_map > 0.5

In [ ]:
plt.imshow(prob_map.squeeze(0).squeeze(0).sum(-1).cpu().numpy())